In [8]:
from datetime import datetime
from elasticsearch import Elasticsearch

es = Elasticsearch()

es.indices.create(index='my-index2', ignore=400)



{'acknowledged': True}

In [9]:
es.index(index="my-index2", doc_type="test-type", id=42,
    body={"any": "data", "timestamp": datetime.now()})



{'_id': '42',
 '_index': 'my-index2',
 '_type': 'test-type',
 '_version': 1,
 'created': True}

In [10]:
es.get(index="my-index", doc_type="test-type", 
    id=42)['_source']


{'any': 'data', 'timestamp': '2015-10-08T18:18:29.871411'}

In [17]:
import requests
import pprint

pp = pprint.PrettyPrinter(indent=4)

# make sure ES is running
res = requests.get('http://localhost:9200')
# pp.pprint(res.content)
print(res.content)


b'{\n  "status" : 200,\n  "name" : "jamie_01_test",\n  "cluster_name" : "EStest1_jamie-test",\n  "version" : {\n    "number" : "1.7.2",\n    "build_hash" : "e43676b1385b8125d647f593f7202acbd816e8ec",\n    "build_timestamp" : "2015-09-14T09:49:53Z",\n    "build_snapshot" : false,\n    "lucene_version" : "4.10.4"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [18]:
# connect to my cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [21]:
# using star wars api (swapi), iterate over swapi people documents and index them
import json
r = requests.get('http://localhost:9200')
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/' + str(i))
    string_body = json.loads(r.content.decode("utf-8-sig"))
    es.index(index='sw', doc_type='people', id=i, body=string_body)
    i=i+1
    
print(i)

18


In [22]:
es.get(index='sw', doc_type='people', id=5)

{'_id': '5',
 '_index': 'sw',
 '_source': {'birth_year': '19BBY',
  'created': '2014-12-10T15:20:09.791000Z',
  'edited': '2014-12-20T21:17:50.315000Z',
  'eye_color': 'brown',
  'films': ['http://swapi.co/api/films/6/',
   'http://swapi.co/api/films/3/',
   'http://swapi.co/api/films/2/',
   'http://swapi.co/api/films/1/'],
  'gender': 'female',
  'hair_color': 'brown',
  'height': '150',
  'homeworld': 'http://swapi.co/api/planets/2/',
  'mass': '49',
  'name': 'Leia Organa',
  'skin_color': 'light',
  'species': ['http://swapi.co/api/species/1/'],
  'starships': [],
  'url': 'http://swapi.co/api/people/5/',
  'vehicles': ['http://swapi.co/api/vehicles/30/']},
 '_type': 'people',
 '_version': 1,
 'found': True}

In [25]:
es.search(index="sw", body={"query": {"match": {"name": "Darth Vader"}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '4',
    '_index': 'sw',
    '_score': 1.4965448,
    '_source': {'birth_year': '41.9BBY',
     'created': '2014-12-10T15:18:20.704000Z',
     'edited': '2014-12-20T21:17:50.313000Z',
     'eye_color': 'yellow',
     'films': ['http://swapi.co/api/films/6/',
      'http://swapi.co/api/films/3/',
      'http://swapi.co/api/films/2/',
      'http://swapi.co/api/films/1/'],
     'gender': 'male',
     'hair_color': 'none',
     'height': '202',
     'homeworld': 'http://swapi.co/api/planets/1/',
     'mass': '136',
     'name': 'Darth Vader',
     'skin_color': 'white',
     'species': ['http://swapi.co/api/species/1/'],
     'starships': ['http://swapi.co/api/starships/13/'],
     'url': 'http://swapi.co/api/people/4/',
     'vehicles': []},
    '_type': 'people'}],
  'max_score': 1.4965448,
  'total': 1},
 'timed_out': False,
 'took': 4}

In [26]:
# more like 'prefix' here: 
# https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html

es.search(index='sw', body={'query': {'prefix': {'name': 'lu'}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'sw',
    '_score': 1.0,
    '_source': {'birth_year': '19BBY',
     'created': '2014-12-09T13:50:51.644000Z',
     'edited': '2014-12-20T21:17:56.891000Z',
     'eye_color': 'blue',
     'films': ['http://swapi.co/api/films/7/',
      'http://swapi.co/api/films/6/',
      'http://swapi.co/api/films/3/',
      'http://swapi.co/api/films/2/',
      'http://swapi.co/api/films/1/'],
     'gender': 'male',
     'hair_color': 'blond',
     'height': '172',
     'homeworld': 'http://swapi.co/api/planets/1/',
     'mass': '77',
     'name': 'Luke Skywalker',
     'skin_color': 'fair',
     'species': ['http://swapi.co/api/species/1/'],
     'starships': ['http://swapi.co/api/starships/12/',
      'http://swapi.co/api/starships/22/'],
     'url': 'http://swapi.co/api/people/1/',
     'vehicles': ['http://swapi.co/api/vehicles/14/',
      'http://swapi.co/api/vehicles/30/']},
    '_type': 'peop

In [29]:
es.search(index='sw', body={'query': {'fuzzy_like_this_field': {
                'name': {'like_text': 'jaba', 'max_query_terms': 5}
            }}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '16',
    '_index': 'sw',
    '_score': 0.8465736,
    '_source': {'birth_year': '600BBY',
     'created': '2014-12-10T17:11:31.638000Z',
     'edited': '2014-12-20T21:17:50.338000Z',
     'eye_color': 'orange',
     'films': ['http://swapi.co/api/films/4/',
      'http://swapi.co/api/films/3/',
      'http://swapi.co/api/films/1/'],
     'gender': 'hermaphrodite',
     'hair_color': 'n/a',
     'height': '175',
     'homeworld': 'http://swapi.co/api/planets/24/',
     'mass': '1,358',
     'name': 'Jabba Desilijic Tiure',
     'skin_color': 'green-tan, brown',
     'species': ['http://swapi.co/api/species/5/'],
     'starships': [],
     'url': 'http://swapi.co/api/people/16/',
     'vehicles': []},
    '_type': 'people'}],
  'max_score': 0.8465736,
  'total': 1},
 'timed_out': False,
 'took': 19}

In [30]:
# more tutorials here:
# http://blog.tryolabs.com/categories/elasticsearch/page/2/

In [33]:
# facets:
# https://www.elastic.co/guide/en/elasticsearch/reference/0.90/search-facets.html

es2 = Elasticsearch()

es.index(index='articles', doc_type='article', 
         id=1, body={"title": "One", "tags": ["foo"]})
es.index(index='articles', doc_type='article', 
         id=2, body={"title": "Two", "tags": ["foo", "bar"]})
es.index(index='articles', doc_type='article', 
         id=3, body={"title": "Three", "tags": ["foo", "bar", "baz"]})
es.index(index='articles', doc_type='article',
         id=10, body={"title": "Ten", "tags": []})



{'_id': '10',
 '_index': 'articles',
 '_type': 'article',
 '_version': 1,
 'created': True}

In [35]:
# search with facets:
es.search(index="articles", 
          body={
            "query": {"query_string": {"query": "T*"}},
            "facets": {
                "tags": {"terms": {"field": "tags"}}
            } # has 1 'missing' facet bc Ten is a document
              # that has no value for the faceted field
          }
         )

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'facets': {'tags': {'_type': 'terms',
   'missing': 1,
   'other': 0,
   'terms': [{'count': 2, 'term': 'foo'},
    {'count': 2, 'term': 'bar'},
    {'count': 1, 'term': 'baz'}],
   'total': 5}},
 'hits': {'hits': [{'_id': '2',
    '_index': 'articles',
    '_score': 1.0,
    '_source': {'tags': ['foo', 'bar'], 'title': 'Two'},
    '_type': 'article'},
   {'_id': '3',
    '_index': 'articles',
    '_score': 1.0,
    '_source': {'tags': ['foo', 'bar', 'baz'], 'title': 'Three'},
    '_type': 'article'},
   {'_id': '10',
    '_index': 'articles',
    '_score': 1.0,
    '_source': {'tags': [], 'title': 'Ten'},
    '_type': 'article'}],
  'max_score': 1.0,
  'total': 3},
 'timed_out': False,
 'took': 3}

In [39]:
# using elasticsearch-dsl

from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q

client = Elasticsearch()

s = Search(using=client, index="sw") \
    .filter('term', category='search') \
    .query('match', title='T*') \
    .query(~Q('match', description='beta'))
    
s.aggs.bucket('per_tag', 'terms', field='tags') \
    .metric('max_lines', 'max', field='lines')
    
response = s.execute()

for hit in response:
    print(hit.meta.score, hit.title)
    
for tag in response.aggregations.per_tag.buckets:
    print(tag.key, tag.max_lines.value)